# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 16:09:30,688 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from pyspark.sql import functions as F

 # SQL  (1 query)

In [3]:
click = se.read.option("sep", "\t").option("header",True).csv("hdfs:/data/clickstream.csv")
click.registerTempTable("click")
click.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [4]:
clicksql = se.sql("""
with error_table as 
    (select CONCAT_WS(':', user_id, session_id) as user_id_session_id,
            timestamp
     from click
     where event_type like '%error%')
,error_table_without_dub as 
     (select user_id_session_id,
             min(timestamp) as timestamp
      from error_table
      group by user_id_session_id)
,click_mod as 
    (select CONCAT_WS(':', user_id, session_id) as user_id_session_id,
            user_id,
            session_id,
            event_type,
            event_page,
            timestamp
     from click )
select route,count(route) as count   from      (select collect_list( distinct(event_page)) as route,
               user_id_session_id
        from (select
                 click_mod.user_id_session_id,
                 click_mod.user_id,
                 click_mod.session_id,
                 click_mod.event_type,
                 click_mod.event_page,
                 click_mod.timestamp,
                 error_table_without_dub.timestamp ,
                 Case when click_mod.timestamp > error_table_without_dub.timestamp then 0 else 1 end  as Line_without_Error 
             from
                 click_mod left outer join error_table_without_dub on click_mod.user_id_session_id=error_table_without_dub.user_id_session_id 
 ) 
        where Line_without_Error =1
        group by user_id_session_id)
group by route
order by count DESC
""")

In [5]:
a=clicksql.withColumn("route", F.array_join(clicksql.route, "-")).collect()

In [6]:
# total number of routes
count=0
for x in a:
    count+=x['count']
count

48523

In [7]:
# top 30 routes
MyDic={}
for x in a [:30]:
    MyDic[x['route']]=x['count']
MyDic


{'main': 8184,
 'main-archive': 1212,
 'main-rabota': 1155,
 'main-internet': 979,
 'main-bonus': 941,
 'main-news': 834,
 'main-tariffs': 733,
 'main-online': 635,
 'main-vklad': 549,
 'main-archive-rabota': 231,
 'main-rabota-archive': 223,
 'main-internet-rabota': 189,
 'main-bonus-archive': 186,
 'main-bonus-rabota': 185,
 'main-archive-internet': 184,
 'main-news-rabota': 182,
 'main-rabota-bonus': 179,
 'main-rabota-internet': 177,
 'main-internet-archive': 174,
 'main-archive-bonus': 173,
 'main-archive-news': 173,
 'main-rabota-news': 169,
 'main-news-archive': 153,
 'main-internet-bonus': 152,
 'main-archive-tariffs': 146,
 'main-internet-news': 140,
 'main-news-internet': 133,
 'main-archive-online': 132,
 'main-tariffs-archive': 132,
 'main-tariffs-internet': 128}

 # SQL  (2 query)

In [8]:
click = se.read.option("sep", "\t").option("header",True).csv("hdfs:/data/clickstream.csv")
click.registerTempTable("click")

In [9]:
clicksql1 = se.sql("""
with error_table as 
    (select CONCAT_WS(':', user_id, session_id) as user_id_session_id,
            timestamp
     from click
     where event_type like '%error%')
,error_table_without_dub as 
     (select user_id_session_id,
             min(timestamp) as timestamp
      from error_table
      group by user_id_session_id)
,click_mod as 
    (select CONCAT_WS(':', user_id, session_id) as user_id_session_id,
            user_id,
            session_id,
            event_type,
            event_page,
            timestamp
     from click )
 (select collect_list(distinct(event_page)) as route,
               user_id_session_id
        from (select
                 click_mod.user_id_session_id,
                 click_mod.user_id,
                 click_mod.session_id,
                 click_mod.event_type,
                 click_mod.event_page,
                 click_mod.timestamp,
                 error_table_without_dub.timestamp ,
                 Case when click_mod.timestamp > error_table_without_dub.timestamp then 0 else 1 end  as Line_without_Error 
             from
                 click_mod left outer join error_table_without_dub on click_mod.user_id_session_id=error_table_without_dub.user_id_session_id 
 ) 
        where Line_without_Error =1
        group by user_id_session_id)

""")

In [10]:
clicksql1.registerTempTable("clicksql1")

In [11]:
clicksql2 = se.sql("""
select 
     route,count(route) as count  
from 
     clicksql1
group by route
order by count DESC
""")

In [12]:
a=clicksql2.withColumn("route", F.array_join(clicksql2.route, "-")).collect()

In [13]:
# total number of routes
count=0
for x in a:
    count+=x['count']
count

48523

In [14]:
# top 30 routes
MyDic={}
for x in a [:30]:
    MyDic[x['route']]=x['count']
MyDic

{'main': 8184,
 'main-archive': 1208,
 'main-rabota': 1155,
 'main-internet': 977,
 'main-bonus': 941,
 'main-news': 830,
 'main-tariffs': 731,
 'main-online': 635,
 'main-vklad': 549,
 'main-archive-rabota': 232,
 'main-rabota-archive': 223,
 'main-internet-rabota': 190,
 'main-bonus-archive': 186,
 'main-archive-internet': 185,
 'main-bonus-rabota': 183,
 'main-news-rabota': 182,
 'main-rabota-bonus': 179,
 'main-rabota-internet': 175,
 'main-archive-bonus': 173,
 'main-internet-archive': 173,
 'main-archive-news': 170,
 'main-rabota-news': 169,
 'main-news-archive': 154,
 'main-internet-bonus': 151,
 'main-archive-tariffs': 145,
 'main-internet-news': 140,
 'main-news-internet': 134,
 'main-archive-online': 134,
 'main-tariffs-archive': 133,
 'main-tariffs-internet': 127}

# DF

In [15]:
clickstream_df= se.read.option("sep", "\t").option("header",True).csv("hdfs:/data/clickstream.csv")
clickstream_df_mod =clickstream_df.withColumn("user_id_session_id",(F.concat_ws(":",clickstream_df["user_id"],clickstream_df['session_id'])))

clickstream_df_mod.filter(clickstream_df_mod['session_id']==874).filter(clickstream_df_mod["user_id"]==0).show()
clickstream_df_mod.filter(clickstream_df_mod['session_id']==351).filter(clickstream_df_mod["user_id"]==3542).show()

+-------+----------+----------+----------+----------+------------------+
|user_id|session_id|event_type|event_page| timestamp|user_id_session_id|
+-------+----------+----------+----------+----------+------------------+
|      0|       874|      page|      main|1696371064|             0:874|
|      0|       874|     event|      main|1696372696|             0:874|
|      0|       874|     event|      main|1696373564|             0:874|
|      0|       874|      page|    rabota|1696374894|             0:874|
|      0|       874|     event|    rabota|1696377393|             0:874|
|      0|       874|      page|    online|1696378229|             0:874|
|      0|       874|     event|    online|1696378928|             0:874|
+-------+----------+----------+----------+----------+------------------+



+-------+----------+------------------+----------+----------+------------------+
|user_id|session_id|        event_type|event_page| timestamp|user_id_session_id|
+-------+----------+------------------+----------+----------+------------------+
|   3542|       351|              page|      main|1700088698|          3542:351|
|   3542|       351|              page|    online|1700089082|          3542:351|
|   3542|       351|             event|    online|1700089785|          3542:351|
|   3542|       351|             event|    online|1700089869|          3542:351|
|   3542|       351|yofherrorloHoSTKYY|    online|1700090430|          3542:351|
|   3542|       351|             event|    online|1700091352|          3542:351|
|   3542|       351|              page|   tariffs|1700091477|          3542:351|
|   3542|       351|              page|   archive|1700091659|          3542:351|
|   3542|       351|             event|   archive|1700091739|          3542:351|
|   3542|       351|        

In [16]:
# total number of routes
c=clickstream_df_mod.groupby(clickstream_df_mod['user_id_session_id']).agg(F.countDistinct('user_id_session_id').alias('count')).collect()
count=0
for x in c:
    count+=x['count']
count

48523

In [17]:
# find line with errors
clickstream_df_errors= (
    clickstream_df_mod
    .filter(clickstream_df_mod['event_type'].like('%error%'))
    .withColumnRenamed('timestamp', 'timestamp_error')
    .withColumnRenamed('user_id_session_id', 'user_id_session_id_error')
    .select(['timestamp_error','user_id_session_id_error'])
    .dropDuplicates(['user_id_session_id_error'])
)

clickstream_df_errors.filter(clickstream_df_errors['user_id_session_id_error']=='0:874').show()
clickstream_df_errors.filter(clickstream_df_errors['user_id_session_id_error']=='3542:351').show()
clickstream_df_errors.count()

+---------------+------------------------+
|timestamp_error|user_id_session_id_error|
+---------------+------------------------+
+---------------+------------------------+



+---------------+------------------------+
|timestamp_error|user_id_session_id_error|
+---------------+------------------------+
|     1700090430|                3542:351|
+---------------+------------------------+



14569

In [18]:
# join base table with the table_errors
clickstream_df_ready=(clickstream_df_mod
     .join(clickstream_df_errors,clickstream_df_mod.user_id_session_id == clickstream_df_errors.user_id_session_id_error,'left_outer')
     .withColumn("Line_without_Error", F.when(clickstream_df_mod.timestamp >= clickstream_df_errors.timestamp_error , 0).otherwise(1)) 
    )

clickstream_df_ready.filter(clickstream_df_ready['user_id_session_id']=='0:874').show()
clickstream_df_ready.filter(clickstream_df_ready['user_id_session_id']=='3542:351').show()

+-------+----------+----------+----------+----------+------------------+---------------+------------------------+------------------+
|user_id|session_id|event_type|event_page| timestamp|user_id_session_id|timestamp_error|user_id_session_id_error|Line_without_Error|
+-------+----------+----------+----------+----------+------------------+---------------+------------------------+------------------+
|      0|       874|      page|      main|1696371064|             0:874|           null|                    null|                 1|
|      0|       874|     event|      main|1696372696|             0:874|           null|                    null|                 1|
|      0|       874|     event|      main|1696373564|             0:874|           null|                    null|                 1|
|      0|       874|      page|    rabota|1696374894|             0:874|           null|                    null|                 1|
|      0|       874|     event|    rabota|1696377393|             0:8

+-------+----------+------------------+----------+----------+------------------+---------------+------------------------+------------------+
|user_id|session_id|        event_type|event_page| timestamp|user_id_session_id|timestamp_error|user_id_session_id_error|Line_without_Error|
+-------+----------+------------------+----------+----------+------------------+---------------+------------------------+------------------+
|   3542|       351|              page|      main|1700088698|          3542:351|     1700090430|                3542:351|                 1|
|   3542|       351|              page|    online|1700089082|          3542:351|     1700090430|                3542:351|                 1|
|   3542|       351|             event|    online|1700089785|          3542:351|     1700090430|                3542:351|                 1|
|   3542|       351|             event|    online|1700089869|          3542:351|     1700090430|                3542:351|                 1|
|   3542|    

In [19]:
# make table with relevant lines
clickstream_df_ready1=(clickstream_df_ready                    
.filter(clickstream_df_ready.Line_without_Error==1)
.select(['user_id','session_id','event_type','event_page','user_id_session_id','timestamp'])
                       
)

clickstream_df_ready1.filter(clickstream_df_ready1['user_id_session_id']=='0:874').show()
clickstream_df_ready1.filter(clickstream_df_ready1['user_id_session_id']=='3542:351').show()

+-------+----------+----------+----------+------------------+----------+
|user_id|session_id|event_type|event_page|user_id_session_id| timestamp|
+-------+----------+----------+----------+------------------+----------+
|      0|       874|      page|      main|             0:874|1696371064|
|      0|       874|     event|      main|             0:874|1696372696|
|      0|       874|     event|      main|             0:874|1696373564|
|      0|       874|      page|    rabota|             0:874|1696374894|
|      0|       874|     event|    rabota|             0:874|1696377393|
|      0|       874|      page|    online|             0:874|1696378229|
|      0|       874|     event|    online|             0:874|1696378928|
+-------+----------+----------+----------+------------------+----------+



+-------+----------+----------+----------+------------------+----------+
|user_id|session_id|event_type|event_page|user_id_session_id| timestamp|
+-------+----------+----------+----------+------------------+----------+
|   3542|       351|      page|      main|          3542:351|1700088698|
|   3542|       351|      page|    online|          3542:351|1700089082|
|   3542|       351|     event|    online|          3542:351|1700089785|
|   3542|       351|     event|    online|          3542:351|1700089869|
+-------+----------+----------+----------+------------------+----------+



In [20]:
# make a table with routes
clickstream_df_ready2=(clickstream_df_ready1                  
  .groupby(clickstream_df_ready1.user_id_session_id)                
  .agg(F.collect_list("event_page").alias("event_page_list"))
                  
)


clickstream_df_ready2.filter(clickstream_df_ready2['user_id_session_id']=='0:874').show()
clickstream_df_ready2.filter(clickstream_df_ready2['user_id_session_id']=='3542:351').show()

+------------------+--------------------+
|user_id_session_id|     event_page_list|
+------------------+--------------------+
|             0:874|[main, main, main...|
+------------------+--------------------+



+------------------+--------------------+
|user_id_session_id|     event_page_list|
+------------------+--------------------+
|          3542:351|[main, online, on...|
+------------------+--------------------+



In [21]:
def dropdupl(raw):
    new_list=[]
    origin_list=raw
    for x in origin_list:
        if x not in new_list:

            new_list.append(x)
        else:
            if new_list[-1]!=x:
                new_list.append(x)
    return "-".join(new_list)

dropdupl = se.udf.register("dropdupl", dropdupl)


In [22]:
# delete duplicated words from the routes and change format
clickstream_df_ready3= (clickstream_df_ready2               
   .select(dropdupl(clickstream_df_ready2.event_page_list).alias('event_page_list1'))
)

clickstream_df_ready3.show(100)


+--------------------+
|    event_page_list1|
+--------------------+
|                main|
|main-news-archive...|
|                main|
|main-bonus-archiv...|
|main-bonus-news-r...|
|                main|
|         main-rabota|
|       main-internet|
|main-archive-onli...|
|                main|
|                main|
|  main-tariffs-bonus|
| main-archive-rabota|
|main-archive-inte...|
|main-internet-tar...|
|         main-rabota|
|main-online-bonus...|
|        main-archive|
|main-archive-tari...|
|archive-rabota-ta...|
|main-bonus-intern...|
|main-bonus-news-i...|
|                main|
|main-news-rabota-...|
|main-bonus-rabota...|
|       main-internet|
|main-bonus-rabota...|
|        main-archive|
|                main|
|main-vklad-main-r...|
|     main-vklad-news|
|main-internet-rab...|
|main-internet-mai...|
|        main-archive|
|                main|
|main-rabota-onlin...|
|        main-archive|
|         main-rabota|
|main-archive-bonu...|
|main-news-bonus-t...|
|main-news-

In [23]:
# count routes
clickstream_df_ready4= (clickstream_df_ready3                   
      .groupby(clickstream_df_ready3.event_page_list1)
      .count()
      .orderBy('count', ascending=False)
) 

clickstream_df_ready4.show(30)

+--------------------+-----+
|    event_page_list1|count|
+--------------------+-----+
|                main| 8170|
|        main-archive| 1102|
|         main-rabota| 1045|
|       main-internet|  893|
|          main-bonus|  867|
|           main-news|  764|
|        main-tariffs|  677|
|         main-online|  581|
|          main-vklad|  516|
| main-rabota-archive|  168|
| main-archive-rabota|  166|
|  main-bonus-archive|  141|
|   main-rabota-bonus|  138|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  133|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  125|
|main-rabota-internet|  123|
|main-internet-arc...|  123|
|  main-archive-bonus|  119|
|main-tariffs-inte...|  114|
| main-internet-bonus|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|  main-internet-news|  102|
|main-archive-tariffs|  102|
|main-tariffs-archive|  102|
| main-archive-online|   93|
+--------------------+-----+
only showing t

In [24]:
a =clickstream_df_ready4.collect()

In [25]:
# total number of routes
count=0
for x in a:
    count+=x['count']
count

48522

In [26]:
# top 30 routes
MyDic={}
for x in a [:30]:
    MyDic[x['event_page_list1']]=x['count']
MyDic

{'main': 8170,
 'main-archive': 1103,
 'main-rabota': 1045,
 'main-internet': 891,
 'main-bonus': 868,
 'main-news': 766,
 'main-tariffs': 676,
 'main-online': 583,
 'main-vklad': 517,
 'main-rabota-archive': 168,
 'main-archive-rabota': 165,
 'main-bonus-archive': 140,
 'main-rabota-bonus': 139,
 'main-bonus-rabota': 135,
 'main-news-rabota': 134,
 'main-archive-internet': 133,
 'main-rabota-news': 129,
 'main-internet-rabota': 128,
 'main-archive-news': 125,
 'main-internet-archive': 124,
 'main-rabota-internet': 123,
 'main-archive-bonus': 119,
 'main-tariffs-internet': 114,
 'main-internet-bonus': 114,
 'main-news-archive': 113,
 'main-news-internet': 109,
 'main-archive-tariffs': 103,
 'main-internet-news': 102,
 'main-tariffs-archive': 102,
 'main-archive-main': 94}

As we can see that Spark works incorrect . In 'clickstream_df_ready4.show(30)'  we can see that 'main-archive ' route has 1102 users  while in 'clickstream_df_ready4.collect()' has 1103 users

# RDD

In [27]:
from collections import OrderedDict

In [28]:
def dropdupl1(raw):
    new_list=[]
    origin_list=raw
    for x in origin_list:
        if x not in new_list:

            new_list.append(x)
        else:
            if new_list[-1]!=x:
                new_list.append(x)
    return "-".join(new_list)

In [29]:
click_rdd= se.read.option("sep", "\t").option("header",True).csv("hdfs:/data/clickstream.csv")
click_rdd_error=(click_rdd.rdd
   .filter(lambda x: x.event_type!='page')
   .filter(lambda x: x.event_type!='event')
   .map (lambda x : (x.user_id+':'+x.session_id, x.timestamp))
   .groupByKey()
   .mapValues(min)
)

click_rdd_filtered=(click_rdd.rdd
   .map (lambda x : (x.user_id+':'+x.session_id,x.timestamp))
   .leftOuterJoin(click_rdd_error)
   .filter(lambda a:  a[1][1]==None or a[1][0] <  a[1][1])
   .map(lambda a : (a[0]+'+'+a[1][0],1))
)


#click_rdd_filtered.collect()


In [30]:
click_rdd_mod=(click_rdd.rdd
   .map (lambda x : (x.user_id+':'+x.session_id+'+'+x.timestamp,x.event_page))
   .leftOuterJoin(click_rdd_filtered)
   .filter(lambda a: a[1][1] != None)            
   .map(lambda a: (a[0].split('+')[0],a[1][0]))
   .groupByKey()
   .mapValues(list)
   .map(lambda a: (dropdupl1(a[1]),1) )
   .reduceByKey(lambda a,b: a+b)
   .sortBy(lambda a: a[1] ,ascending=False )
   #.map(lambda a: sum(a[1]) )
         
) 

a= click_rdd_mod.collect()



In [31]:
# total number of routes
count=0
for x in a:
    count+=x[1]
count

48522

In [32]:
# top 30 routes
MyDic={}
for x in a [:30]:
    MyDic[x[0]]=x[1]
MyDic

{'main': 8184,
 'main-archive': 291,
 'main-rabota': 272,
 'archive-main': 262,
 'rabota-main': 242,
 'main-internet': 240,
 'main-bonus': 237,
 'main-news': 215,
 'bonus-main': 203,
 'main-archive-main': 194,
 'internet-main': 187,
 'main-rabota-main': 186,
 'news-main': 177,
 'archive-main-archive': 176,
 'rabota-main-rabota': 173,
 'main-tariffs': 171,
 'main-internet-main': 164,
 'main-online': 162,
 'internet-main-internet': 161,
 'main-bonus-main': 151,
 'tariffs-main': 145,
 'main-vklad': 139,
 'bonus-main-bonus': 132,
 'online-main': 129,
 'main-news-main': 128,
 'tariffs-main-tariffs': 124,
 'vklad-main': 122,
 'main-tariffs-main': 117,
 'main-online-main': 115,
 'news-main-news': 112}

In [33]:
a

[('main', 8184),
 ('main-archive', 291),
 ('main-rabota', 272),
 ('archive-main', 262),
 ('rabota-main', 242),
 ('main-internet', 240),
 ('main-bonus', 237),
 ('main-news', 215),
 ('bonus-main', 203),
 ('main-archive-main', 194),
 ('internet-main', 187),
 ('main-rabota-main', 186),
 ('news-main', 177),
 ('archive-main-archive', 176),
 ('rabota-main-rabota', 173),
 ('main-tariffs', 171),
 ('main-internet-main', 164),
 ('main-online', 162),
 ('internet-main-internet', 161),
 ('main-bonus-main', 151),
 ('tariffs-main', 145),
 ('main-vklad', 139),
 ('bonus-main-bonus', 132),
 ('online-main', 129),
 ('main-news-main', 128),
 ('tariffs-main-tariffs', 124),
 ('vklad-main', 122),
 ('main-tariffs-main', 117),
 ('main-online-main', 115),
 ('news-main-news', 112),
 ('main-vklad-main', 87),
 ('main-archive-main-archive', 81),
 ('vklad-main-vklad', 74),
 ('online-main-online', 73),
 ('main-rabota-main-rabota', 71),
 ('archive-main-archive-main', 64),
 ('rabota-main-rabota-main', 64),
 ('main-intern